In [1]:
import pandas as pd
import igraph as ig
import random
import itertools
import operator
import pickle
import time
import threading
import concurrent.futures

In [2]:
InHostUserMap = {}
with open('InHostUserMap.pkl', 'rb') as pickle_file:
    InHostUserMap = pickle.load(pickle_file)

In [3]:
InHostSrcMap = {}
with open('InHostSrcMap.pkl', 'rb') as pickle_file:
    InHostSrcMap = pickle.load(pickle_file)

In [4]:
InHostUsrSrcMap = {}
with open('InHostUsrSrcMap.pkl', 'rb') as pickle_file:
    InHostUsrSrcMap = pickle.load(pickle_file)

In [5]:
OutHostDstMap = {}
with open('OutHostDstMap.pkl', 'rb') as pickle_file:
    OutHostDstMap = pickle.load(pickle_file)

In [6]:
OutHostUserMap = {}
with open('OutHostUserMap.pkl', 'rb') as pickle_file:
    OutHostUserMap = pickle.load(pickle_file)

In [7]:
OutHostUsrDstMap = {}
with open('OutHostUsrDstMap.pkl', 'rb') as pickle_file:
    OutHostUsrDstMap = pickle.load(pickle_file)

In [8]:
## user_id, item_id, timestamp, state_label, comma_seperated_list_of_features
auth = pd.read_csv("../auth_ntlm.txt", header=None, names=['time', 'src_t', 'user', 'src', 'dst', 'auth_t', 'log_t', 'auth_o', 's_f'])

In [9]:
authlog = auth[['src', 'dst', 'time', 'user']]

In [10]:
authlog['ID_usr'] = 0.0
authlog['ID_src'] = 0.0
authlog['ID_usr_src'] = 0.0

#authlog['OD_usr'] = 0.0
#authlog['OD_dst'] = 0.0
#authlog['OD_usr_dst'] = 0.0

#authlog['IDAF_usr'] = 0.0
#authlog['IDAF_src'] = 0.0
#authlog['IDAF_usr_src'] = 0.0

#authlog['ODAF_usr'] = 0.0
#authlog['ODAF_dst'] = 0.0
#authlog['ODAF_usr_dst'] = 0.0

#authlog['IDAFSTD_usr'] = 0.0
#authlog['IDAFSTD_src'] = 0.0
#authlog['IDAFSTD_usr_src'] = 0.0

#authlog['ODAFSTD_usr'] = 0.0
#authlog['ODAFSTD_dst'] = 0.0
#authlog['ODAFSTD_usr_dst'] = 0.0

/tmp/ipykernel_29545/2196784689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authlog['ID_usr'] = 0.0
/tmp/ipykernel_29545/2196784689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authlog['ID_src'] = 0.0
/tmp/ipykernel_29545/2196784689.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [11]:
authlog

,src,dst,time,user,ID_usr,ID_src,ID_usr_src
0,C1250,C586,1,ANONYMOUS LOGON@C586,0.0,0.0,0.0
1,C2944,C2944,1,U898@DOM1,0.0,0.0,0.0
2,C4615,C457,2,ANONYMOUS LOGON@C457,0.0,0.0,0.0
3,C2734,C586,2,ANONYMOUS LOGON@C586,0.0,0.0,0.0
4,C3896,C586,2,ANONYMOUS LOGON@C586,0.0,0.0,0.0
...,...,...,...,...,...,...,...
49341295,C659,C1065,5011199,C659$@DOM1,0.0,0.0,0.0
49341296,C9458,C1015,5011199,C9458$@DOM1,0.0,0.0,0.0
49341297,C419,C419,5011199,U129@DOM1,0.0,0.0,0.0
49341298,C9458,C1015,5011199,U3887@DOM1,0.0,0.0,0.0


--

In [12]:
def Average(lst):
    return sum(lst) / len(lst)

In [13]:
def Stddev(lst):
    mean = Average(lst)
    variance = sum([((x - mean) ** 2) for x in lst]) / len(lst)
    stddev = variance ** 0.5
    return stddev

--

In [14]:
## Algorithm 7
def Calculate_IDusr(dst):
    InHostUserMap_list = list(InHostUserMap.keys())
    if dst in InHostUserMap_list:
        return len(list(InHostUserMap[dst].values()))
    else:
        return 0

In [15]:
## Algorithm 8
def Calculate_IDsrc(dst):
    InHostSrcMap_list = list(InHostSrcMap.keys())
    if dst in InHostSrcMap_list:
        return len(list(InHostSrcMap[dst].values()))
    else:
        return 0

In [16]:
## Algorithm 9
def Calculate_IDusr_src(dst):
    InHostUsrSrcMap_list = list(InHostUsrSrcMap.keys())
    if dst in InHostUsrSrcMap_list:
        return len(list(InHostUsrSrcMap[dst].values()))
    else:
        return 0

--

In [17]:
## Algorithm 10
def Calculate_ODusr(src):
    if src in list(OutHostUserMap.keys()):
        return len(list(OutHostUserMap[src].values()))
    else:
        return 0

In [18]:
## Algorithm 11
def Calculate_ODdst(src):
    if src in list(OutHostDstMap.keys()):
        return len(list(OutHostDstMap[src].values()))
    else:
        return 0

In [19]:
## Algorithm 12
def Calculate_ODusr_dst(src):
    if src in list(OutHostUsrDstMap.keys()):
        return len(list(OutHostUsrDstMap[src].values()))
    else:
        return 0

--

In [20]:
## Algorithm 13
def Calculate_IDAFusr(dst):
    UserFrequency = []
    if dst in list(InHostUserMap.keys()):
        for user in list(InHostUserMap[dst].keys()):
            UserFreq = Average(InHostUserMap[dst][user].values())
            UserFrequency.append(UserFreq)
        return Average(UserFrequency)
    else:
        return 0

In [21]:
## Algorithm 14
def Calculate_IDAFsrc(dst):
    SrcFrequency = []
    if dst in list(InHostSrcMap.keys()):
        for src in list(InHostSrcMap[dst].keys()):
            SrcFreq = Average(InHostSrcMap[dst][src].values())
            SrcFrequency.append(SrcFreq)
        return Average(SrcFrequency)
    else:
        return 0

In [22]:
## Algorithm 15
def Calculate_IDAFusr_src(dst):
    UsrSrcFrequency = []
    if dst in list(InHostUsrSrcMap.keys()):
        for usrsrc in list(InHostUsrSrcMap[dst].keys()):
            UsrSrcFreq = Average(InHostUsrSrcMap[dst][usrsrc].values())
            UsrSrcFrequency.append(UsrSrcFreq)
        return Average(UsrSrcFrequency)
    else:
        return 0

--

In [23]:
## Algorithm 16
def Calculate_ODAFusr(src):
    UserFrequency = []
    if src in list(OutHostUserMap.keys()):
        for user in list(OutHostUserMap[src].keys()):
            UsrFreq = Average(OutHostUserMap[src][user].values())
            UserFrequency.append(UsrFreq)
        return Average(UserFrequency)
    else:
        return 0

In [24]:
## Algorithm 17
def Calculate_ODAFdst(src):
    DstFrequency = []
    if src in list(OutHostDstMap.keys()):
        for dst in list(OutHostDstMap[src].keys()):
            dstFreq = Average(OutHostDstMap[src][dst].values())
            DstFrequency.append(dstFreq)
        return Average(DstFrequency)
    else:
        return 0

In [25]:
## Algorithm 18
def Calculate_ODAFusr_dst(src):
    UsrDstFrequency = []
    if src in list(OutHostUsrDstMap.keys()):
        for usrdst in list(OutHostUsrDstMap[src].keys()):
            usrdstFreq = Average(OutHostUsrDstMap[src][usrdst].values())
            UsrDstFrequency.append(usrdstFreq)
        return Average(UsrDstFrequency)
    else:
        return 0

--

In [26]:
## Algorithm 19
def Calculate_IDAFSTDusr(dst):
    UserFrequency = []
    if dst in list(InHostUserMap.keys()):
        for user in list(InHostUserMap[dst].keys()):
            UserFreq = Average(InHostUserMap[dst][user].values())
            UserFrequency.append(UserFreq)
        return Stddev(UserFrequency)
    else:
        return 0

In [27]:
## Algorithm 20
def Calculate_IDAFSTDsrc(dst):
    SrcFrequency = []
    if dst in list(InHostSrcMap.keys()):
        for src in list(InHostSrcMap[dst].keys()):
            SrcFreq = Average(InHostSrcMap[dst][src].values())
            SrcFrequency.append(SrcFreq)
        return Stddev(SrcFrequency)
    else:
        return 0

In [28]:
## Algorithm 21
def Calculate_IDAFSTDusr_src(dst):
    UsrSrcFrequency = []
    if dst in list(InHostUsrSrcMap.keys()):
        for usersrc in list(InHostUserMap[dst].keys()):
            UsrSrcFreq = Average(InHostUserMap[dst][usersrc].values())
            UsrSrcFrequency.append(UsrSrcFreq)
        return Stddev(UsrSrcFrequency)
    else:
        return 0

--

In [29]:
## Algorithm 22
def Calculate_ODAFSTDusr(src):
    UserFrequency = []
    if src in list(OutHostUserMap.keys()):
        for user in list(OutHostUserMap[src].keys()):
            UserFreq = Average(OutHostUserMap[src][user].values())
            UserFrequency.append(UserFreq)
        return Stddev(UserFrequency)
    else:
        return 0

In [30]:
## Algorithm 23
def Calculate_ODAFSTDdst(src):
    DstFrequency = []
    if src in list(OutHostDstMap.keys()):
        for dst in list(OutHostDstMap[src].keys()):
            DstFreq = Average(OutHostDstMap[src][dst].values())
            DstFrequency.append(DstFreq)
        return Stddev(DstFrequency)
    else:
        return 0

In [31]:
## Algorithm 24
def Calculate_ODAFSTDusr_dst(src):
    UsrDstFrequency = []
    if src in list(OutHostUsrDstMap.keys()):
        for userdst in list(OutHostUsrDstMap[src].keys()):
            UsrDstFreq = Average(OutHostUsrDstMap[src][userdst].values())
            UsrDstFrequency.append(UsrDstFreq)
        return Stddev(UsrDstFrequency)
    else:
        return 0

--

In [32]:
def features(authlog):
    count = 1
    init = time.time()
    print(init)
    print("=======================")
    for i, event in authlog.iterrows():
        if i/(1000000*count) == 1:
            count +=1
            print("index = ", i)
            now = time.time()
            lap = now-init
            print(lap)
            init = time.time()
            
        authlog.at[i,'ID_usr'] = Calculate_IDusr(event['dst'])
        #authlog.at[i,'ID_src'] = Calculate_IDsrc(event['dst'])
        #authlog.at[i,'ID_usr_src'] = Calculate_IDusr_src(event['dst'])
        
        #with concurrent.futures.ThreadPoolExecutor() as executor:
        #    future = executor.submit(Calculate_IDusr, event['dst'])
        #    authlog.at[i,'ID_usr'] = future.result()
            
        #with concurrent.futures.ThreadPoolExecutor() as executor:
        #    future = executor.submit(Calculate_IDsrc, event['dst'])
        #    authlog.at[i,'ID_usr'] = future.result()
            
        #with concurrent.futures.ThreadPoolExecutor() as executor:
        #    future = executor.submit(Calculate_IDusr_src, event['dst'])
        #    authlog.at[i,'ID_usr'] = future.result()
            
        #authlog.at[i,'ID_usr'] = threading.Thread(target=Calculate_IDusr, args=(event['dst'],))
        #authlog.at[i,'ID_src'] = threading.Thread(target=Calculate_IDsrc, args=(event['dst'],))
        #authlog.at[i,'ID_usr_src'] = threading.Thread(target=Calculate_IDusr_src, args=(event['dst'],))
        
        #authlog.at[i,'OD_usr'] = Calculate_ODusr(event['src'])
        #authlog.at[i,'OD_dst'] = Calculate_ODdst(event['src'])
        #authlog.at[i,'OD_usr_dst'] = Calculate_ODusr_dst(event['src'])
        
        #authlog.at[i,'IDAF_usr'] = Calculate_IDAFusr(event['dst'])
        #authlog.at[i,'IDAF_src'] = Calculate_IDAFsrc(event['dst'])
        #authlog.at[i,'IDAF_usr_src'] = Calculate_IDAFusr_src(event['dst'])
        
        #authlog.at[i,'ODAF_usr'] = Calculate_ODAFusr(event['src'])
        #authlog.at[i,'ODAF_dst'] = Calculate_ODAFdst(event['src'])
        #authlog.at[i,'ODAF_usr_dst'] = Calculate_ODAFusr_dst(event['src'])
        
        #authlog.at[i,'IDAFSTD_usr'] = Calculate_IDAFSTDusr(event['dst'])
        #authlog.at[i,'IDAFSTD_src'] = Calculate_IDAFSTDsrc(event['dst'])
        #authlog.at[i,'IDAFSTD_usr_src'] = Calculate_IDAFSTDusr_src(event['dst'])
        
        #authlog.at[i,'ODAFSTD_usr'] = Calculate_ODAFSTDusr(event['src'])
        #authlog.at[i,'ODAFSTD_dst'] = Calculate_ODAFSTDdst(event['src'])
        #authlog.at[i,'ODAFSTD_usr_dst'] = Calculate_ODAFSTDusr_dst(event['src'])
        
    return authlog

In [33]:
#authlog = authlog.head(100)

In [ ]:
new_feat_authlog = features(authlog)
pickle.dump(new_feat_authlog, open("new_feat_authlog_cp1-1.pkl", "wb")) 

1656222023.6065958
index =  1000000
446.85743284225464
index =  2000000
439.97337675094604
index =  3000000
447.3260667324066
index =  4000000
448.658171415329
index =  5000000
444.2423975467682
index =  6000000
446.1448874473572
index =  7000000
441.410133600235
index =  8000000
439.2510709762573
index =  9000000
439.6958501338959
index =  10000000
438.8812289237976
index =  11000000
434.6940631866455
index =  12000000
431.7829134464264
index =  13000000
442.4417505264282
index =  14000000
444.4004123210907


In [42]:
new_feat_authlog_cp1_1 = {}
with open('new_feat_authlog_cp1-1.pkl', 'rb') as pickle_file:
    new_feat_authlog_cp1_1 = pickle.load(pickle_file)

In [43]:
new_feat_authlog_cp1_1

,src,dst,time,user,ID_usr,ID_src,ID_usr_src
0,C1250,C586,1,ANONYMOUS LOGON@C586,11518.0,0.0,0.0
1,C2944,C2944,1,U898@DOM1,4.0,0.0,0.0
2,C4615,C457,2,ANONYMOUS LOGON@C457,11089.0,0.0,0.0
3,C2734,C586,2,ANONYMOUS LOGON@C586,11518.0,0.0,0.0
4,C3896,C586,2,ANONYMOUS LOGON@C586,11518.0,0.0,0.0
...,...,...,...,...,...,...,...
49341295,C659,C1065,5011199,C659$@DOM1,11251.0,0.0,0.0
49341296,C9458,C1015,5011199,C9458$@DOM1,970.0,0.0,0.0
49341297,C419,C419,5011199,U129@DOM1,20.0,0.0,0.0
49341298,C9458,C1015,5011199,U3887@DOM1,970.0,0.0,0.0
